In [ ]:
# Importando Bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tqdm import tqdm
import random
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [ ]:
# Definição do SVM SGD
class LinearSVM:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.n = x.shape[1]
        
    def _stochastic_subgradient_descent(self):
        epochs = 10000
        learning_rate = 1e-8
        w = np.zeros(self.n)
        
        for epoch in tqdm(range(1, epochs+1)):
            
            X = shuffle(self.x) #arrumar o shuffle
            Y = shuffle(self.y)
            
            for i, xi in enumerate(X):
                yi = Y[i]
                if(yi*(np.dot(w.T, xi))<=1):
                    w = (1 - learning_rate)*w + learning_rate*yi*xi
                else:
                    w = (1 - learning_rate)*w
                
        return w
    
    def fit(self):
        w = self._stochastic_subgradient_descent()
        norm = np.linalg.norm(w)
        b = self.y - np.dot(self.x, w)
        #normalize
        b = b.sum() / b.size
        self.w, self.b = w / norm, b / norm

    
    def predict(self, x):
        #w, b = self._fit()
        y = np.sign(np.dot(self.w, x.T) + self.b * np.ones(x.shape[0]))
        return y

In [ ]:
# Dados para Treino
x_train =  np.array ([ \
        [1.00, 0.31], [1.10, 0.20], [1.12, 0.42], [1.55, 0.75], [1.11, 0.53], #-1
        [1.34, 0.32], [0.19, 4.30], [0.51, 2.91], [0.24, 2.58], [1.17, 0.89], #-1
        [1.95, 0.44], [2.12, 0.25], [1.06, 0.21], [0.33, 0.20], [0.46, 0.59], #-1
        [0.79, 1.33], [0.76, 1.91], [1.34, 1.64], [0.21, 1.18], [0.70, 2.53], #-1
        [0.54, 2.23], [1.55, 1.29], [1.46, 0.51], [0.18, 3.37], [0.48, 3.54], #-1
        [2.51, 1.10], [2.33, 1.03], [2.21, 1.52], [1.92, 2.05], [2.24, 3.06], # 1
        [2.71, 3.35], [0.53, 4.60], [0.80, 4.08], [1.01, 3.35], [1.68, 3.95], # 1
        [1.38, 3.86], [2.14, 2.56], [2.16, 1.76], [1.39, 2.84], [1.71, 3.27], # 1
        [1.73, 1.82], [2.28, 1.09], [3.00, 1.33], [2.54, 0.34], [3.16, 0.35], # 1
        [3.12, 0.88], [2.59, 2.27], [2.40, 3.29], [2.31, 2.92], [1.54, 2.38]  # 1
    ])

y_train = np.array([ \
        [-1], [-1], [-1], [-1], [-1],
        [-1], [-1], [-1], [-1], [-1],
        [-1], [-1], [-1], [-1], [-1],
        [-1], [-1], [-1], [-1], [-1],
        [-1], [-1], [-1], [-1], [-1],
        [1], [1], [1], [1], [1], 
        [1], [1], [1], [1], [1],
        [1], [1], [1], [1], [1], 
        [1], [1], [1], [1], [1],
        [1], [1], [1], [1], [1]
    ])

In [ ]:
# Definição dos dados de treino: cada ponto "-" corresponde ao label -1 e cada ponto "+" corresponde ao label 1
x1_train_pos = []
x2_train_pos = []
x1_train_neg = [] 
x2_train_neg = []
X1trpos, X2trpos,X1trneg, X2trneg=[],[],[],[]

for i in range(len(x_train)):
    if y_train[i] == -1:
        X1trneg, X2trneg = x_train[i][0], x_train[i][1]
        x1_train_neg.append(X1trneg) 
        x2_train_neg.append(X2trneg)
        
    else:
        X1trpos, X2trpos = x_train[i][0], x_train[i][1]
        x1_train_pos.append(X1trpos)
        x2_train_pos.append(X2trpos)
        
        
#Plot do gráfico treino
def plot_treino():
    fig, ax = plt.subplots(figsize=(8,6))
    plt.title("SVM Treino")
    plt.xlabel('$X_1$')
    plt.ylabel('$X_2$') 
    plt.scatter(x1_train_pos,x2_train_pos, s=120, marker='+', linewidths=2, label = "$1$", color = 'r')
    plt.scatter(x1_train_neg,x2_train_neg, s=120, marker='_', linewidths=2, label = "$-1$", color = 'b')
    plt.plot([2.4,0.1],[0.0,4.9],'k')
    plt.legend()
    plt.savefig('SVMTrain.png')
plot_treino()

In [ ]:
# Dados para Treino
x_test =  np.array ([ \
        [1.01, 0.82], [0.87, 0.58], [1.63, 0.87], [1.64, 0.32], [0.78, 1.03], #-1
        [1.11, 1.39], [0.70, 1.57], [0.22, 1.62], [0.26, 2.15], [1.05, 2.35], #-1
        [0.68, 3.17], [0.68, 3.17], [0.24, 3.96], [0.18, 0.77], [0.72, 0.26], #-1
        [0.80, 3.64], [1.66, 3.62], [2.19, 3.94], [2.16, 3.48], [1.98, 3.15], # 1
        [2.85, 2.79], [2.13, 2.19], [1.84, 2.83], [1.31, 2.54], [1.77, 1.36], # 1 
        [2.80, 0.98], [2.37, 0.64], [3.41, 2.13], [3.00, 2.00], [1.23, 4.74]  # 1
    ])
# "Embaralhamento" dos dados de teste
x_test_shuffle = np.random.shuffle(x_test)

In [ ]:
x_test

In [ ]:
y_test_s = np.array([1.,1.,-1.,1.,1.,-1.,1.,-1.,-1.,1.,1.,-1.,-1.,1.,1.,
                     -1.,1.,-1.,-1.,1.,1.,-1.,-1.,-1.,-1.,1.,-1.,1.,-1.,1.,])

#eu fui imbecil e fiz isso na mão, mas era mais fácil ter colocado 3 colunas pra cada linha sendo a ultima o label
#e depois cortar e salvar ela. faz isso.

In [ ]:
# Aplicação do SVM Linear
lsvm = LinearSVM(x=x_train, y=y_train)
lsvm.fit()

In [ ]:
y_pred = lsvm.predict(x=x_test)

In [ ]:
# Definição dos dados de teste: cada ponto "-" corresponde ao label -1 e cada ponto "+" corresponde ao label 1
x1_test_pos = []
x2_test_pos = []
x1_test_neg = [] 
x2_test_neg = []
X1tspos, X2tspos,X1tsneg, X2tsneg=[],[],[],[]

for i in range(len(x_test)):
    if y_pred[i] == -1:
        X1tsneg, X2tsneg = x_test[i][0], x_test[i][1]
        x1_test_neg.append(X1tsneg) 
        x2_test_neg.append(X2tsneg)
        
    else:
        X1tspos, X2tspos = x_test[i][0], x_test[i][1]
        x1_test_pos.append(X1tspos)
        x2_test_pos.append(X2tspos)
        
#Plot do gráfico teste
def plot_pred():
    fig, ax = plt.subplots(figsize=(8,6))
    plt.title("SVM Teste")
    plt.xlabel('$X_1$')
    plt.ylabel('$X_2$') 
    plt.scatter(x1_test_pos,x2_test_pos, s=120, marker='+', linewidths=2, label = "$1$", color = 'r')
    plt.scatter(x1_test_neg,x2_test_neg, s=120, marker='_', linewidths=2, label = "$-1$", color = 'b')
    plt.plot([2.4,0.1],[0.0,4.9],'k')
    plt.legend()
    plt.savefig('SVMTest.png')
plot_pred()

In [ ]:
# Análise da Acurácia do Programa
accuracy_score(y_test_s, y_pred)